# Spectroimaging pixel by pixel

Edited by Louise, 18/06/2020

Edited by Martin, 21/06/2020 (implement FastSimulator)

In this notebook, I try to reproduce what Gabriele has done. Spectroimaging on the galaxy center, compare the reconstruction to the input sky pixel by pixel

In [ ]:
%matplotlib inline

import os
import sys
import glob

# Specific science modules
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
import pickle as pk
from importlib import reload
import time
from matplotlib.legend_handler import HandlerLine2D, HandlerTuple
import matplotlib.ticker as mtick

# Specific qubic modules
from astropy.io import fits
import qubic
from pysimulators import FitsArray
from qubicpack.utilities import Qubic_DataDir
from qubic import QubicSkySim as qss
from qubic.polyacquisition import compute_freq
from qubic import ReadMC as rmc
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
import scipy.constants
#from qubic import mcmc
import mcmc_old as mcmc
import ForegroundsSED as fsed

plt.rc('text',usetex=False)
plt.rc('font', size=16)

import qubic.AnalysisMC as amc

Implementing FastSimulator

In [ ]:
# Dictionary saved during the simulation
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])
#dictfilename = global_dir + '/doc/FastSimulator/FastSimDemo_{}.dict'.format(config)
dictfilename = global_dir + '/dicts/spectroimaging_article.dict'
d150Q = qubic.qubicdict.qubicDict()
d150Q.read_from_file(dictfilename)
d150Q['nf_recon'] = 5
d150Q['nf_sub'] = 5 
d150Q['photon_noise']=True
d150Q['RA_center'] = 0.0
d150Q['DEC_center'] = -57.0
d150Q['effective_duration'] = 2
d150Q['npointings']=10000
d150Q['effective_duration'] = 3

#150 galactic center patch (thereafter GC patch)
d150G = d150Q.copy()
d150G['effective_duration'] = 1

#
# 220 Qubic patch
d220Q = d150Q.copy()
d220Q['filter_nu'] = 220e9
# 220 GC patch
d220G = d220Q.copy()
d150G['effective_duration'] = 1

# Qubic patch - galactic coordinates
centerQ = qubic.equ2gal(d150Q['RA_center'], d150Q['DEC_center'])
# Galactic center patch - galactic coordinates 
centerG = np.array([0,0])
d150G['RA_center'], d150G['DEC_center'] = qubic.gal2equ(centerG[0], centerG[1])
d220G['RA_center'], d220G['DEC_center'] = d150G['RA_center'], d150G['DEC_center']

centers = [centerQ, centerQ, centerG, centerG]
dictionaries = [d150Q, d220Q, d150G, d220G]

##### QubicSkySim instanciation
seed = None
sky_config_220 = {'dust': 'd1'}
d220=d.copy()
d220['filter_nu']=220*1e9
d220['nf_recon']=5
Qubic_sky_220 = qss.Qubic_sky(sky_config_220, d220)
auxmaps220, noiseless220, _, _ = Qubic_sky_220.get_partial_sky_maps_withnoise(spatial_noise=False, 
                                                                              coverage=coverage)

Make coverage..

In [ ]:
#Added by JCH 19/03/2021 -> creatd the files
# 			["doc/FastSimulator/Data/DataFastSimulator_FI150Q_coverage.fits'",
# 			 "doc/FastSimulator/Data/DataFastSimulator_FI220Q_coverage.fits'",
# 			 "doc/FastSimulator/Data/DataFastSimulator_FI150G_coverage.fits'",
# 			 "doc/FastSimulator/Data/DataFastSimulator_FI220G_coverage.fits'" ]
# from the files with the same name but extension .npy that are in. the same directory...

from pysimulators import FitsArray
files = ["doc/FastSimulator/Data/DataFastSimulator_FI150Q_coverage",
         "doc/FastSimulator/Data/DataFastSimulator_FI220Q_coverage",
         "doc/FastSimulator/Data/DataFastSimulator_FI150G_coverage",
         "doc/FastSimulator/Data/DataFastSimulator_FI220G_coverage" ]

for f in files:
    fnpy = global_dir+'/'+f+'.npy'
    ffits = global_dir+'/'+f+'.fits'
    if len(glob.glob(ffits))==0:
        print('{} does not exist, creating it from the npy'.format(ffits))
        if len(glob.glob(fnpy))==0:
            print('npy files {} does exist either...'.format(fnpy))
        else:
            print('Reading {}'.format(fnpy))
            cov = np.load(fnpy)
            print('Writing the fits file')
            FitsArray(cov).save(ffits)


In [ ]:
# NEW  16 Feb 2021
reload(fsed)
regions = ['Qubic_field', 'GalCen_field']
bands = ['150', '220']
nf_recon = dictionaries[0]['nf_recon']
coveragesmaps = fsed.coverage(dictionaries, regions, bands)

In [ ]:
# NEW (16 Feb 2021)
#reload(fsed)
#fsed._plot_covs(regions, bands, coveragesmaps, centers)

In [ ]:
# NEW (16 Feb 2021)
reload(fsed)
#sky_conf = {'synch': 's0'}
#sky_conf = {'dust': 'd1'}
sky_conf_dust = {'dust': 'd1'}
sky_conf_synch = {'synch': 's0'}
sky_conf_all = {'dust': 'd1', 'synch': 's0'}
#sky_conf_cmb = {'CMB':'c1'}

fground_maps_dust = fsed.foreground_signal(dictionaries, sky_conf_dust)
#fground_maps_cmb = fsed.foreground_signal(dictionaries, sky_conf_cmb)
fground_maps_synch = fsed.foreground_signal(dictionaries, sky_conf_synch)
fground_maps = fsed.foreground_signal(dictionaries, sky_conf_all)

In [ ]:
# NEW (16 Feb 2021)
reload(fsed)
nreals = 43
noise = fsed.noise_qss(dictionaries, sky_conf_dust, coveragesmaps, nreals, verbose = False) 


In [ ]:
#NEW (16 feb 2021)
reload(fsed)
#maps, _, stdmaps = fsed.foreground_with_noise(dictionaries, sky_conf_all, regions, bands, nreals)

Look at the maps... okk..

In [ ]:
# NEW (16 Feb 2021)
reload(fsed)
#maskmaps, mapregion = fsed._mask_maps(maps, coveragesmaps, dictionaries[0]['nf_recon'])
#maskmaps_cmb, _ = fsed._mask_maps(fground_maps_cmb, coveragesmaps, nf_recon)
maskmaps_dust, _ = fsed._mask_maps(fground_maps_dust, coveragesmaps, nf_recon)
maskmaps_synch, _ = fsed._mask_maps(fground_maps_synch, coveragesmaps, nf_recon)
maskmaps_all, _ = fsed._mask_maps(fground_maps, coveragesmaps, nf_recon)


## Figure 10 paper 1

In [ ]:
# NEW (17 Feb 2021)
_, nus150, nus_out150, _, _, _ = qubic.compute_freq(dictionaries[0]['filter_nu'] / 1e9,  
                            nf_recon,
                            dictionaries[0]['filter_relative_bandwidth'])
_, nus220, nus_out220, _, _, _ = qubic.compute_freq(dictionaries[1]['filter_nu'] / 1e9,  
                            nf_recon,
                            dictionaries[1]['filter_relative_bandwidth'])

reload(fsed)
ix = 2

MapsArray = [maskmaps_dust[ix], maskmaps_synch[ix], maskmaps_all[ix]]
fsed._plot_exampleSED(dictionaries[ix], centers[ix], nus_out150, MapsArray, mapsarray = True, 
                     DeltaTheta = 1, DeltaPhi = 0, savefig = True)


In [ ]:
add_dust = np.zeros((4,2,2,5))
add_synch = np.zeros((4,2,2,5))
add_all = np.zeros((4,2,2,5))
stk = 1
totalpixs = [26411, 27237, 26310, 27123]
for reg in range(4):
    tot = totalpixs[reg]
    for stk in [0,1]:
        if reg == 0: #150Q
            if stk == 0:
                lims = np.array([[0.37,4],[0.6,19]])
            else:
                lims = np.array([[-0.045,-0.75],[0.08,0.7]])
        elif reg == 1: #220Q
            if stk == 0:
                lims = np.array([[0.205,12],[0.34,50]])
            else:
                lims = np.array([[-0.025,-2.3],[0.045,2.1]])
        elif reg == 2: #150G
            if stk == 0:
                lims = np.array([[1.7,1e2],[1.25e1,3e3]])
            else:
                lims = np.array([[-0.35,-8],[0.48,9]])
        elif reg == 3: #220G
            if stk == 0:
                lims = np.array([[0.95,3e2],[7,6.8e3]])
            else:
                lims = np.array([[-0.20,-25],[0.27,27]])

        for j, limlim in enumerate(lims):
            cov_lims = limlim[0]
            cov_lim = limlim[1]
            fig , ax = plt.subplots(nrows = 3, ncols = 5, figsize = (14,10))
            for fr in range(5):
                if stk == 0:
                    mmm = np.copy(maskmaps_dust[reg,fr, :, stk])
                    mmsk = maskmaps_dust[reg,fr, : ,stk] > cov_lim
                    mmm[~mmsk] = hp.UNSEEN

                    mmm_s = np.copy(maskmaps_synch[reg,fr, :, stk])
                    mmsk_s = maskmaps_synch[reg,fr, : ,stk] > cov_lims
                    mmm_s[~mmsk_s] = hp.UNSEEN

                    mmm_a = np.copy(maskmaps_all[reg,fr, :, stk])
                    mmsk_a = maskmaps_all[reg,fr, : ,stk] > cov_lim
                    mmm_a[~mmsk_a] = hp.UNSEEN

                else:
                    mmm1, mmm2 = np.copy(maskmaps_dust[reg,fr, :, stk]), np.copy(maskmaps_dust[reg,fr, :, stk+1])
                    mmm = np.sqrt(mmm1**2 + mmm2**2)
                    mmsk1, mmsk2 = maskmaps_dust[reg,fr, : ,stk] > cov_lim, maskmaps_dust[reg,fr, : ,stk+1] > cov_lim
                    mmsk = mmsk1 * mmsk2
                    mmm[~mmsk] = hp.UNSEEN
                    #print(np.min(mmm[mmsk]), np.sum(coveragesTD[0] > 0)) 
                    mmm_s1, mmm_s2 = np.copy(maskmaps_synch[reg,fr, :, stk]), np.copy(maskmaps_synch[reg,fr, :, stk+1])
                    mmm_s = np.sqrt(mmm_s1**2 + mmm_s2**2)
                    mmsk_s1, mmsk_s2 = maskmaps_synch[reg,fr, : ,stk] > cov_lims, maskmaps_synch[reg,fr, : ,stk+1] > cov_lims
                    mmsk_s = mmsk_s1 * mmsk_s2
                    mmm_s[~mmsk_s] = hp.UNSEEN
                    #print(np.min(mmm_s[mmsk_s]))
                    mmm_a1, mmm_a2 = np.copy(maskmaps_all[reg,fr, :, stk]), np.copy(maskmaps_all[reg,fr, :, stk+1])
                    mmm_a = np.sqrt(mmm_a1**2 + mmm_a2**2)
                    mmsk_a1, mmsk_a2 = maskmaps_all[reg,fr, : ,stk] > cov_lim, maskmaps_all[reg,fr, : ,stk+1] > cov_lim
                    mmsk_a = mmsk_a1 * mmsk_a2
                    mmm_a[~mmsk_a] = hp.UNSEEN

                ax[0,fr].cla()
                plt.axes(ax[0,fr])
                hp.gnomview(mmm, reso = 14, rot = centers[0], 
                            title = "Polvo galáctico" if fr == 2 else " ",
                            max = 6e3 if stk == 0 else 35, hold = True)
                add_dust[reg,stk,j] =np.sqrt(np.mean(mmm[mmsk]**2)) 
                if fr == 2: print("Region coverage (dust) {}".format(sum(mmsk)/tot ))
               
                ax[1,fr].cla()
                plt.axes(ax[1,fr])
                hp.gnomview(mmm_s, reso = 14, rot = centers[0], 
                            title = "Sincrotrón" if fr == 2 else " ", 
                            max = 30 if stk == 0 else 9, hold = True)
                add_synch[reg,stk,j] = np.sqrt(np.mean(mmm_s[mmsk_s]**2)) 
                if fr == 2: print("Region coverage (synch) {}".format(sum(mmsk_s)/tot ))

                plt.axes(ax[2,fr])
                hp.gnomview(mmm_a, reso = 14, rot = centers[0], 
                            title = "Polvo galáctico + sincrotrón" if fr == 2 else " ",
                            max = 6e3 if stk == 0 else 35, hold = True)
                add_all[reg,stk,j] = np.sqrt(np.mean(mmm_a[mmsk_a]**2)) 
                if fr == 2: print("Region coverage (dust+synch) {}".format(sum(mmsk_a)/tot ))
            plt.tight_layout()
            #plt.savefig("RegionSky_SED{}".format(j))

In [ ]:
plt.figure(figsize = (8,6))
for reg in range(4):
    for stk in [0,1]:
        plt.fill_between(nus_out150 if reg in [0,2] else nus_out220, 
                         y1 = add_dust[reg,stk,0], y2 = add_dust[reg,stk,1],
                         color = 'b' if stk == 0 else 'g', alpha = 0.6, 
                         linestyle = "--", lw = 4,
                        label = "Temperatura" if stk == 0 else "Polarización")

        plt.fill_between(nus_out150 if reg in [0,2] else nus_out220, 
                         y1 = add_synch[reg,stk,0], y2 = add_synch[reg,stk,1],
                         color = 'b' if stk == 0 else 'g', alpha = 0.6, 
                         linestyle = "--", lw = 4)

        plt.fill_between(nus_out150 if reg in [0,2] else nus_out220, 
                         y1 = add_all[reg,stk,0], y2 = add_all[reg,stk,1],
                           color = 'r', alpha = 0.8)
        plt.ylabel(r"$rms(\nu)~[\mu K_{\rm CMB}]$")# if stk == 0 else r"$P(\nu)~[\mu K_{\rm CMB}]$", fontsize = 18)
        plt.xlabel(r"$\nu [GHz]$", fontsize = 18)
        plt.yscale("log")

        #plt.ylim(3e-1,5e4)
        #plt.legend(loc = "upper center", ncol = 2)
plt.tight_layout()
#plt.savefig("rms_Comp-Dust_synch-FI")

In [ ]:
writefiles = False
if writefiles:
    file = open("DataSED/FI-component_data.pk", 'wb')
    pk.dump([dictionaries, centers], file)
    file.close()

### Udgrade

In [ ]:
#NEW (17 Feb 2021)
reload(fsed)
nside_new = 64

t0 = time.time()
#fgr_map_cmb_ud = fsed.udgrade_maps(fground_maps_cmb, noise, nside_new, 
#                                    nf_recon, nreals, return_with_noise = False)
fgr_map_dust_ud = fsed.udgrade_maps(fground_maps_dust, noise, nside_new, 
                                    nf_recon, nreals, return_with_noise = False)
fgr_map_synch_ud = fsed.udgrade_maps(fground_maps_synch, noise, nside_new, 
                                     nf_recon, nreals, return_with_noise = False)
maps_ud, _, fgr_map_ud, noise_ud_i = fsed.udgrade_maps(fground_maps, noise, nside_new, 
                                                       nf_recon, nreals,return_with_noise = True)
#_, covmask = fsed._mask_maps(maps, coveragesmaps, nf_recon)
_, covmask = fsed._mask_maps(fground_maps, coveragesmaps, nf_recon)
cov_ud = hp.ud_grade(covmask, nside_new)
Cp = []
Cp_prime = []
for br in range(len(dictionaries)):
    Cp = amc.get_Cp(noise_ud_i[br][:, :, cov_ud[br], :], verbose = False, doplot = False)
    Cp_prime.append(amc.Cp2Cp_prime_viaCorr(Cp, verbose = False)[1])

for idict in range(len(dictionaries)):
    for iNU in range(nf_recon):
        maps_ud[idict, iNU,~cov_ud[idict],:] = hp.UNSEEN
#print("cov_ud shape", np.shape(cov_ud))   

In [ ]:
bandreg = ["Qubic_Field150FI", "Qubic_Field220FI",
           "GalCen_Field150FI", "GalCen_Field220FI"]
if writefiles:
    for j, idict in enumerate(dictionaries):
        file = open("DataSED/SED_CovarMatrix_config{}_nside{}_nreals{}.pk".format(bandreg[j], 
                                                                                       nside_new,
                                                                                       nreals), "wb")
        pk.dump(Cp_prime[j], file)
        file.close()

In [ ]:
PixPix = lambda p: hp.ang2pix(nside_new, hp.pix2ang(dictionaries[0]['nside'], p)[0], 
                              hp.pix2ang(dictionaries[0]['nside'], p)[1] )

Computing the error bars considering covariance between sub-bands

In [ ]:
# NEW (17feb 2021)
reload(fsed)

plt.figure()
plt.imshow(Cp_prime[2][:, :, 0, 100])
plt.colorbar()

### FIT

In [ ]:
# NEW (17 Feb 2021)
#
#   Choose pixel for regions
#
paper = "1"
if paper == "1":
    pixQ = [hp.ang2pix(dictionaries[0]['nside'], 
                       np.pi / 2 - np.deg2rad(centers[0][1] + 15), np.deg2rad(centers[0][0] - 1  ) ), ]
    pixG = [hp.ang2pix(dictionaries[0]['nside'], 
                       np.pi / 2 - np.deg2rad(centers[2][1] - 6), np.deg2rad(centers[2][0] + 3  ) ), ]

    pixQ_ud = PixPix(pixQ[0])
    pixG_ud = PixPix(pixG[0])
    pixQ_ud = 47149
    pixG_ud = 26754
elif paper == "2":
    pixQ_ud = 10759
    pixG_ud = 5313
# Where the sky pixel is in the reduce format (pixels seen array and not full map)
pixQ_red = np.where(np.where(cov_ud[0] == True)[0] == pixQ_ud)[0][0]
pixG_red = np.where(np.where(cov_ud[2] == True)[0] == pixG_ud)[0][0]

plt.figure(figsize = (10,4))
hp.gnomview(maps_ud[2,-1,:,0], reso = 15,#hold = True, 
            notext = False, title = 'G patch ', sub = (121),
            max = 0.4*np.max(maps_ud[2,-1,:,0]), 
            unit = r'$\mu$K',
            rot = centers[2])
hp.projscatter(hp.pix2ang(nside_new, pixG_ud), marker = '*', color = 'r', s = 200)
hp.gnomview(maps_ud[1,-1,:,0], reso = 15, title = 'Q patch ',
            unit = r'$\mu$K', sub = (122),
            rot = centerQ)
hp.projscatter(hp.pix2ang(nside_new, pixQ_ud), marker = '*', color = 'r', s = 200)
hp.graticule(dpar = 10, dmer = 20, alpha = 0.6)

In [ ]:
def Bnu(nuGHz, temp):
    h = scipy.constants.h
    c = scipy.constants.c
    k = scipy.constants.k
    nu = nuGHz * 1e9
    return 2 * h * nu ** 3 / c ** 2 / (np.exp(h * nu / k / temp ) - 1 )
def func353(x, a, b):
    #a = pars[0]
    #b = pars[1]
    Tdust = 19.6
    bnu = Bnu(x, Tdust)
    return a * 1e18 * bnu * (x / 353) ** (b / 2)


In [ ]:
# NEW (17 Feb 2021)
reload(fsed)
reload(qubic)
reload(mcmc)

nus_out = [nus_out150, nus_out220, nus_out150, nus_out220]
pixs_ud = [pixQ_ud, pixQ_ud, pixG_ud, pixG_ud]
pixs_red = [pixQ_red, pixQ_red, pixG_red, pixG_red]
nus_edge = [nus150, nus220, nus150, nus220]

study = "dust"

if study == "dust":
    FuncModel = fsed.ThermDust_Planck353
    #FuncModel = func353
    p0 = np.array([5e2,2.5])
elif study == "synch":
    FuncModel = fsed.Synchrotron_storja
    #p0 = np.array([1e1,20,-3]) #Planch
    p0 = np.array([1e1,3])
elif study == "dust+synch":
    FuncModel = fsed.DustSynch_model
    p0 = np.array([1e6, 3, 1e4, 3])
    
    
Chi2Model = None#"Chi2Implement"

flatprior = [np.array([1e1,1.5]), np.array([[0.3, 0.3], [0.3, 0.3]])]

Imvals, Isvals, xarr, flat_samples = fsed.foregrounds_run_mcmc(dictionaries, fgr_map_dust_ud, Cp_prime, 
                                                               FuncModel,
                                                nus_out, nus_edge, pixs_ud, 
                                                pixs_red = pixs_red, chi2=Chi2Model, 
                                                samples = 5000, verbose = False, initP0 = p0,
                                                               flatprior = None,)
                                                #plotcovar = True, past_mcmc = True)

In [ ]:
if writefiles:
    file = open("DataSED/data_torun_MCMC_nside{}_nreals{}_pixG{}_pixQ{}.pk".format(nside_new,
                                                                                  nreals,
                                                                                  pixG_ud, pixQ_ud), 'wb')
    pk.dump([dictionaries, fgr_map_dust_ud, Cp_prime, 
             nus_out, nus_edge, pixs_ud, pixs_red], file)
    file.close()                                                

In [ ]:
print(flat_samples.shape)

import getdist
from getdist import plots, MCSamples

for i in range(len(dictionaries)):
    samps = MCSamples(samples=flat_samples[i,:,:])
    g = plots.getSubplotPlotter()
    g.triangle_plot(samps, filled=True,title_limit=2)


In [ ]:
#plt.hist(flat_samples[0,:,0], bins = 20, histtype="step")
savefigs = False

plt.figure(figsize=(12,6))
plt.subplot(121)
plt.xlabel("Amp_d")
plt.ylabel(r"$\beta_d$")
plt.hist2d(flat_samples[0,:,0], flat_samples[0,:,1], bins = 20, cmap = "Purples")
if study == "dust+synch":
    plt.subplot(122)
    plt.xlabel("Amp_s")
    plt.ylabel(r"$\beta_S$")
    plt.hist2d(flat_samples[0,:,2], flat_samples[0,:,3], bins = 20, cmap = "Purples")
    plt.tight_layout()
if savefigs:
    plt.savefig('Figs-FI-SED/March2021/NSIDE{}/hits_param_space_{}_{}_{}_{}_{}.svg'.format(nside_new,
                                                                            FuncModel.__name__,
                                                                            d150Q['nf_recon'],nside_new,
                                                           pixQ_ud, pixG_ud), 
            format = 'svg', bbox_inches='tight')
else:
    plt.show()

In [ ]:
#See pasrametric model
xrange = np.arange(50,1100)
ax = plt.subplot(111)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel(r"$\nu$[GHz]")
if study == "dust":
    ax.set_xticks([50,100,1000])
    ax.set_xticklabels(['50', '100', '1000'])
    ax.set_title ("Free dust")
    ax.plot(xrange, FuncModel(xrange, [100, 1],), 'r-', label = "A = 100, b = 1")
    ax.plot(xrange, FuncModel(xrange, [100, 1.5], ), 'y-', label = "A = 100, b = 1.5")
    ax.plot(xrange, FuncModel(xrange, [100, 1.5], ), 'g-', label = "A = 100, b = 1.5")
    ax.plot(xrange, FuncModel(xrange, [100, 1.5], ), 'c-', label = "A = 100, b = 1")
    ax.plot(xrange, FuncModel(xrange, [100, 2.], ), 'b-', label = "A = 100, b = 2")
elif study == "synch":
    ax.set_title ("Synchrotron")
    #ax.plot(xrange, FuncModel(xrange, [30, 20, - 3]), 'r-', label = "A = 30 nu0 = 20 b = -3")
    #ax.plot(xrange, FuncModel(xrange, [30, 20,- 2]), 'y-', label = "A = 30 nu0 = 20 b = -2")
    #ax.plot(xrange, FuncModel(xrange, [30, 20, - 1]), 'g-', label = "A = 30 nu0 = 20 b = -1")
    ax.plot(xrange, FuncModel(xrange, [30, - 3]), 'r-', label = "A = 30 b = -3")
    ax.plot(xrange, FuncModel(xrange, [30, - 2]), 'y-', label = "A = 30 b = -2")
    ax.plot(xrange, FuncModel(xrange, [30, - 1]), 'g-', label = "A = 30 b = -1")
elif study == "dust+synch":
    ax.plot(xrange, FuncModel(xrange, [100, 1.5, 50, - 3]), 'r-', label = r"$A_{d} =100$ $\beta_{dt} = 1.5$ $A_{s} = 30$ $\beta_{s} = -3$")
    ax.plot(xrange, FuncModel(xrange, [200, 1.5, 50, - 3]), 'b-', label = r"$A_{d} =200$ $\beta_{dt} = 1.5$ $A_{s} = 50$ $\beta_{s} = -3$")
    ax.plot(xrange, FuncModel(xrange, [200, 1.5, 50, - 2]), 'g-', label = r"$A_{d} =200$ $\beta_{dt} = 1.5$ $A_{s} = 50$ $\beta_{s} = -2$")
    ax.plot(xrange, FuncModel(xrange, [200, 5, 50, - 2]), 'y-', label = r"$A_{d} =200$ $\beta_{dt} = 5$ $A_{s} = 50$ $\beta_{s} = -2$")
    
#ax.plot(xrange, FuncModel(xrange, 100, 1.55, T = 23), 'k-', label = "Best fit-Planck")
ax.axvspan(130, 170, ymin = 0, ymax = ax.get_ylim()[-1], color = 'k', alpha = 0.2)
ax.axvspan(210, 260, ymin = 0, ymax = ax.get_ylim()[-1], color = 'k', alpha = 0.2)
ax.legend(fontsize = 10)

In [ ]:
# NEW (17 Feb 2021)
#
# Make a fit for SED values (in each frequency) to match dimenssions between 
# the Energy in the sub-basnds for a given pixel (ndim = nf_recon) and the 
# error coming from the MCMC runs (nsamples//2 samples) 
#
#
reload(fsed)

xSED = [nus_out150, nus_out220, nus_out150, nus_out220]

if study == "dust":
    FuncPoint = fsed.ThermDust_Planck353_pointer
    #FuncPoint = fsed.ThermDust_Planck545_pointer
    initGuess = np.array([1, 3])
    
elif study == "synch":
    FuncPoint = fsed.Synchrotron_storja_pointer
    #FuncPoint = fsed.Synchrotron_Planck_pointer

elif study == "dust+synch":
    FuncPoint = fsed.DustSynch_model_pointer
    initGuess = np.array([1e3, 3, 1e8, 3])
    
ySED_fit, Pmean, Perr = fsed.make_fit_SED(xSED, xarr, Imvals, Isvals,
                                          FuncPoint, fgr_map_ud, pixs_ud, nf_recon, 
                                          initP0 = initGuess, 
                                          maxfev = 15000)

In [ ]:
plt.plot(np.arange(2500), np.sqrt(Imvals[0,:,1]**2 + Imvals[0,:,1]**2), 'r-')
plt.fill_between(np.arange(2500), y1 = np.sqrt(Imvals[0,:,1]**2 +Imvals[0,:,2]**2) - \
                 np.sqrt(Isvals[0,:,1]**2 + Isvals[0,:,2]**2), 
                 y2 = np.sqrt(Imvals[0,:,1]**2 + Imvals[0,:,2]**2) + \
                 np.sqrt(Isvals[0,:,1]**2 + Isvals[0,:,2]**2) )

In [ ]:
if writefiles:
    for j, idict in enumerate(dictionaries):
        file = open("DataSED/MCMC_run_{}nside{}_nreals{}_pixG{}_pixQ{}.pk".format(dictionaries[0]["config"],
                                                                      nside_new,
                                                                    nreals,
                                                                    pixG_ud, pixQ_ud), "wb")
        pk.dump([Imvals, Isvals, xarr, ySED_fit, Pmean, Perr, pixs_ud], file)
        file.close()

    file = open("DataSED/ForegroundMaps_{}nside{}.pk".format(dictionaries[0]["config"],
                                                                      nside_new), "wb")
    pk.dump([fgr_map_dust_ud, fgr_map_synch_ud, fgr_map_ud, maps_ud, cov_ud], file)
    file.close()
    

In [ ]:
# NEW (17 feb 2021)
savefigs = False

RESO = 15
capsize = 3
fonts = 16
plt.rc('font', size = 16)

fig,ax = plt.subplots(nrows = 1, ncols = 4,figsize = (19,4.5), gridspec_kw = {'wspace': 0.4})
ax = ax.ravel()
plt.subplots_adjust(wspace = 0.1)
# Plotting
# Dust galactic center
#t0, = ax[0].plot(nus_out150, fgr_map_dust_ud[2,:,pixs_ud[2],0], ls = '', 
#           marker = 'd', color = 'g',alpha = 0.5, label = 'Dust')
#ax[0].plot(nus_out220, fgr_map_dust_ud[3,:,pixs_ud[2],0], marker = 'd', ls = '', color = 'g', alpha = 0.5)
#Synch galactic center
#t1, = ax[0].plot(nus_out150, fgr_map_synch_ud[2,:,pixs_ud[2],0], ls = '', 
#           marker = 's', color = 'g', alpha = 0.5, label = 'Synchrotron')
#ax[0].plot(nus_out220, fgr_map_synch_ud[3,:,pixs_ud[2],0], marker = 's', ls = '', color = 'g', alpha = 0.5)
#Two components
p1, = ax[0].plot(nus_out150, fgr_map_ud[2, :, pixs_ud[2], 0], 'ro', label = 'Input sky')
p2, = ax[0].plot(nus_out220, fgr_map_ud[3, :, pixs_ud[2], 0], 'bo')


e1 = ax[0].fill_between(xarr[2,:], y1 = ySED_fit[2,:,0] - Isvals[2, :, 0], 
                                y2 = ySED_fit[2, :, 0] + Isvals[2, :, 0], 
                 color = 'r', alpha = 0.3, label = '68% C.L.')
e2 = ax[0].fill_between(xarr[3, :], y1 = ySED_fit[3, :, 0] - Isvals[3, :, 0], 
                        y2 = ySED_fit[3, :, 0] + Isvals[3, :, 0], 
                   color = 'b', alpha = 0.3)

# Dust galactic center
#ax[2].plot(nus_out150, fgr_map_dust_ud[0,:,pixs_ud[0],0], ls = '', 
#           marker = 'd', color = 'g',alpha = 0.5, label = 'Dust')
#ax[2].plot(nus_out220, fgr_map_dust_ud[1,:,pixs_ud[0],0], marker = 'd', ls = '', color = 'g', alpha = 0.5)
#Synch galactic center
#ax[2].plot(nus_out150, fgr_map_synch_ud[0,:,pixs_ud[0],0], ls = '', 
#           marker = 's', color = 'g', alpha = 0.5, label = 'Synchrotron')
#ax[2].plot(nus_out220, fgr_map_synch_ud[1,:,pixs_ud[0],0], marker = 's', ls = '', color = 'g', alpha = 0.5)

ax[2].plot(nus_out150, fgr_map_ud[0, :, pixs_ud[0], 0], 'ro')
ax[2].plot(nus_out220, fgr_map_ud[1, :, pixs_ud[0], 0], 'bo')
ax[2].fill_between(xarr[0, :], y1 = ySED_fit[0, :, 0] - Isvals[0, :, 0], 
                   y2 = ySED_fit[0, :, 0] + Isvals[0, :, 0], 
                   color = 'r', alpha = 0.3)
ax[2].fill_between(xarr[1, :], y1 = ySED_fit[1, :, 0] - Isvals[1, :, 0], 
                   y2 = ySED_fit[1, :, 0] + Isvals[1, :, 0], 
                   color = 'b', alpha = 0.3)

# Settings
greyscale = 0.1
ax[2].axvspan(nus150[-1], nus220[0],color='k',alpha = greyscale)
ax[0].axvspan(nus150[-1], nus220[0],color='k',alpha = greyscale)
xlim = ax[0].get_xlim()
ylim = ax[0].get_ylim()
xlim2 = ax[2].get_xlim()
ylim2 = ax[2].get_ylim()
ax[0].axvspan(xlim[0], nus150[0], color = 'k', alpha = greyscale)
ax[0].axvspan(nus220[-1], xlim[-1], color = 'k', alpha = greyscale)

ax[2].axvspan(xlim2[0], nus150[0], color = 'k', alpha = greyscale)
ax[2].axvspan(nus220[-1], xlim2[-1], color = 'k', alpha = greyscale)

logscale = True
if logscale:
    ax[0].set_yscale("log")
    ax[2].set_yscale("log")
    ax[0].set_xscale("log")
    ax[2].set_xscale("log")

ax[0].set_xticks([150, 220], ['150','220'])
ax[0].xaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax[0].xaxis.set_minor_formatter(mtick.ScalarFormatter())
ax[0].tick_params(axis = "both", which = "both",
                  direction='in',width=1.3,)
ax[0].set_xlim(xlim)
ax[0].set_ylim(ylim)

ax[2].set_xticks([150, 220], ['150','220'])
ax[2].xaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax[2].xaxis.set_minor_formatter(mtick.ScalarFormatter())
ax[2].tick_params(axis = "both", which = "both",
                  direction='in',width=1.3,)

ax[2].set_xlim(xlim2)
ax[2].set_ylim(ylim2)

ax[0].grid(which = "both")
ax[2].grid(which = "both")
l = ax[0].legend([(p1, p2), (e1, e2)], ['Input sky', '68% C.L.'], numpoints=1, 
                 loc = 2, fontsize = 12,
               handler_map={tuple: HandlerTuple(ndivide=None)})

ax[0].set_title('GC patch - {} year'.format(dictionaries[0]['effective_duration']), fontsize = fonts)
ax[0].set_ylabel(r'$I(\nu)$ [$\mu$K$_{CMB}$]',fontsize = fonts)
ax[0].set_xlabel(r'$\nu$[GHz]',fontsize = fonts)

ax[2].set_title('QUBIC patch - {} years'.format(dictionaries[0]['effective_duration']),fontsize = fonts)
ax[2].set_ylabel(r'$I(\nu)$ [$\mu$K$_{CMB}$]',fontsize = fonts)
ax[2].set_xlabel(r'$\nu$[GHz]',fontsize = fonts)

# Displaying maps
ax[1].cla()
plt.axes(ax[1])
hp.gnomview(maps_ud[2, -1, :, 0], reso = 15,hold = True, 
            notext = True, title = ' ',
            min = 0,
            max = 0.4*np.max(maps_ud[2, -1, :, 0]), 
            unit = r'$\mu$K$_{CMB}$',
            rot = centers[2])
hp.projscatter(hp.pix2ang(nside_new, pixs_ud[2]), marker = '*', color = 'r', s = 180)
dpar = 10
dmer = 20
#Watch out, the names are wrong (change it)
mer_coordsG = [centers[2][0] - dmer, centers[2][0], centers[2][0] + dmer]
long_coordsG = [centers[2][1] - 2*dpar, centers[2][1] - dpar, 
                centers[2][1], centers[2][1] + dpar, centers[2][1] + 2 * dpar]
#paralels
for ilong in long_coordsG:
    plt.text(np.deg2rad(mer_coordsG[0] - 14), 1.1*np.deg2rad(ilong), 
             r'{}$\degree$'.format(ilong))
#meridians
for imer in mer_coordsG:
    if imer < 0:
        jmer = imer + 360
        ip, dp = divmod(jmer/15,1)
    else:
        ip, dp = divmod(imer/15,1)
    if imer == 0:
        plt.text(-np.deg2rad(imer + 3), np.deg2rad(long_coordsG[-1] + 6), 
             r'{}$\degree$'.format(int(ip) ))
    else:
        plt.text(-np.deg2rad(imer + 3), np.deg2rad(long_coordsG[-1] + 6), 
             r'{}$\degree$'.format(imer))
             #r'{}h{}m'.format(int(ip), int(round(dp*60))))
#hp.projtext(mer_coordsG[1] + 2, long_coordsG[0] - 6, '$l$',  color = 'k', lonlat=True)
#hp.projtext(mer_coordsG[2] + 12.5, long_coordsG[2] - 1, '$b$', rotation = 90, color = 'k', lonlat=True)
plt.text(0.5,0.13, '$l$', transform=ax[1].transAxes)
plt.text(-0.3,0.57, '$b$', transform=ax[1].transAxes)

ax[3].cla()
plt.axes(ax[3])
hp.gnomview(maps_ud[1, -1, :, 0], reso = 15, hold = True, 
            notext = True, title = ' ',
            unit = r'$\mu$K$_{CMB}$',
            min = 0,
            max = 0.4*np.max(maps_ud[1, -1, :, 0]), 
            rot = centerQ)
hp.projscatter(hp.pix2ang(nside_new, pixQ_ud),marker = '*', color = 'r', s = 180)

mer_coordsQ = [centers[1][0] - dmer, centers[0][0]+0, centers[0][0] + dmer]
long_coordsQ = [centers[0][1] - 2*dpar, centers[0][1] - dpar, centers[0][1], 
                centers[0][1] + dpar, centers[0][1] + 2 * dpar]
#paralels
for ilong in long_coordsQ:
    plt.text(np.deg2rad(mer_coordsQ[0]-360+29), 1.1*np.deg2rad(ilong+58), r'{:.0f}$\degree$'.format(ilong),)
#meridians
for imer in mer_coordsQ:
    ip, dp = divmod(imer/15,1)
    plt.text(-np.deg2rad(imer-360+48), np.deg2rad(long_coordsQ[-1]+58+7), 
         r'{:.1f}$\degree$'.format(imer))
         #r'{}h{}m'.format(int(ip), int(round(dp*60))))
#hp.projtext(mer_coordsQ[0] , long_coordsQ[0] - 6, r'$l$',  color = 'k', lonlat=True)
#hp.projtext(mer_coordsQ[2] - 360, long_coordsQ[2] + 10, r'$b$', rotation = 90, color = 'k', lonlat=True)
plt.text(0.5,0.13, '$l$', transform = ax[3].transAxes)
plt.text(-0.3,0.57, '$b$', transform = ax[3].transAxes)

hp.graticule(dpar = dpar, dmer = dmer, alpha = 0.6, verbose = False)

plt.tight_layout()
#savefigs = True
if savefigs:
    plt.savefig('Figs-FI-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixQ{}_pixG{}_Intensity_log.svg'.format(nside_new,
                                                                            FuncModel.__name__,
                                                                            d150Q['nf_recon'],nside_new,
                                                           pixQ_ud, pixG_ud), 
            format = 'svg', bbox_inches='tight')
    plt.savefig('Figs-FI-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixQ{}_pixG{}_Intensity_log.pdf'.format(nside_new,
                                                                                FuncModel.__name__,
                                                                               d150Q['nf_recon'],nside_new,
                                                           pixQ_ud, pixG_ud), 
            format = 'pdf', bbox_inches='tight')
    plt.savefig('Figs-FI-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixQ{}_pixG{}_Intensity_log'.format(nside_new,
                                                                        FuncModel.__name__, 
                                                                           d150Q['nf_recon'],
                                                                           nside_new,pixQ_ud, pixG_ud),
           bbox_inches='tight')
else:
    plt.show()

### Polarization

Average realizations and compute std of realizations

In [ ]:
# NEW (17 Feb 2021)

fig,ax = plt.subplots(nrows = 1,ncols = 4, figsize = (19,4.5), gridspec_kw = {'wspace': 0.4})
ax = ax.ravel()
plt.subplots_adjust(wspace=0.1)

# Dust galactic center
t0, = ax[0].plot(nus_out150, np.sqrt(fgr_map_dust_ud[2,:,pixs_ud[2],1] ** 2 + \
                 fgr_map_dust_ud[2,:,pixs_ud[2],2] **2),
           ls = '', marker = 'd', color = 'g',alpha = 0.5, label = 'Dust')
ax[0].plot(nus_out220, np.sqrt(fgr_map_dust_ud[3,:,pixs_ud[2],1] ** 2 + \
                 fgr_map_dust_ud[3,:,pixs_ud[2],2] **2), 
           marker = 'd', ls = '', color = 'g', alpha = 0.5)
#Synch galactic center
t1, = ax[0].plot(nus_out150, np.sqrt(fgr_map_synch_ud[2,:,pixs_ud[2],1] ** 2 + \
                                    fgr_map_synch_ud[2,:,pixs_ud[2],2] ** 2), 
                 ls = '', marker = 's', color = 'g', alpha = 0.5, label = 'Synchrotron')
ax[0].plot(nus_out220, np.sqrt(fgr_map_synch_ud[3,:,pixs_ud[2],1] ** 2 + \
                                    fgr_map_synch_ud[3,:,pixs_ud[2],2] ** 2),
           marker = 's', ls = '', color = 'g', alpha = 0.5)

# Plotting
p1, = ax[0].plot(nus_out150, 
               np.sqrt(fgr_map_ud[2,:,pixs_ud[2],1] ** 2 + fgr_map_ud[2,:,pixs_ud[2],2] ** 2),
               'ro', lw = 3, label = 'Input sky')
p2, = ax[0].plot(nus_out220, 
               np.sqrt(fgr_map_ud[3,:,pixs_ud[3],1] ** 2 + fgr_map_ud[3,:,pixs_ud[3],2] ** 2),
               'bo', lw = 3)

e1 = ax[0].fill_between(xarr[2], y1 = ySED_fit[2, :, 1] - Perr[2], 
                        y2 = ySED_fit[2, :, 1] + Perr[2], 
                   color = 'r', alpha = 0.3, label = '68% C.L. ')
e2 = ax[0].fill_between(xarr[3], y1 = ySED_fit[3, :, 1] - Perr[3], 
                        y2 = ySED_fit[3, :, 1] + Perr[3], 
                   color = 'b', alpha = 0.3)
ax[0].axvspan(nus150[-1], nus220[0], color = 'k', alpha = greyscale)

# Dust galactic center
ax[2].plot(nus_out150, np.sqrt(fgr_map_dust_ud[0,:,pixs_ud[0],1] ** 2 + \
                 fgr_map_dust_ud[0,:,pixs_ud[0],2] **2),
           ls = '', marker = 'd', color = 'g',alpha = 0.5, label = 'Dust')
ax[2].plot(nus_out220, np.sqrt(fgr_map_dust_ud[1,:,pixs_ud[0],1] ** 2 + \
                 fgr_map_dust_ud[1,:,pixs_ud[0],2] **2), 
           marker = 'd', ls = '', color = 'g', alpha = 0.5)
#Synch galactic center
ax[2].plot(nus_out150, np.sqrt(fgr_map_synch_ud[0,:,pixs_ud[0],1] ** 2 + \
                                    fgr_map_synch_ud[0,:,pixs_ud[0],2] ** 2), 
                 ls = '', marker = 's', color = 'g', alpha = 0.5, label = 'Synchrotron')
ax[2].plot(nus_out220, np.sqrt(fgr_map_synch_ud[1,:,pixs_ud[0],1] ** 2 + \
                                    fgr_map_synch_ud[1,:,pixs_ud[0],2] ** 2),
           marker = 's', ls = '', color = 'g', alpha = 0.5)

ax[2].plot(nus_out150, 
               np.sqrt(fgr_map_ud[0,:,pixs_ud[0],1] ** 2 + fgr_map_ud[0, :, pixs_ud[0], 2] ** 2),
               'ro', lw = 3)
ax[2].plot(nus_out220, 
               np.sqrt(fgr_map_ud[1, :, pixs_ud[1], 1] ** 2 + fgr_map_ud[1, :, pixs_ud[1], 2] ** 2),
               'bo', lw = 3)
ax[2].fill_between(xarr[0], y1 = ySED_fit[0, :, 1] - Perr[0], 
                   y2 = ySED_fit[0, :, 1] + Perr[0], 
                   color = 'r', alpha = 0.3)
ax[2].fill_between(xarr[1], y1 = ySED_fit[1, :, 1] - Perr[1], 
                   y2 = ySED_fit[1, :, 1] + Perr[1], 
                   color = 'b', alpha = 0.3)

# Setting
ax[0].set_title('GC patch - {} year'.format(dictionaries[0]['effective_duration']), fontsize = 14)
ax[0].set_ylabel(r'$P(\nu)~[\mu$K]', fontsize = 14)
ax[0].set_xlabel(r'$\nu~[GHz]$', fontsize = 14)
ax[0].legend(loc = 2, fontsize = 12)
ax[0].grid()
ax[2].set_xlabel(r'$\nu~[GHz]$', fontsize = 14)
ax[2].axvspan(nus150[-1], nus220[0], color = 'k', alpha = greyscale)
ax[2].set_ylabel(r'$P(\nu)~[\mu$K]', fontsize = 14)
ax[2].set_title('QUBIC patch - {} years'.format(dictionaries[0]['effective_duration']),fontsize=14)
ax[2].grid()

xlim = ax[0].get_xlim()
ylim = ax[0].get_ylim()
xlim2 = ax[2].get_xlim()
ylim2 = ax[2].get_ylim()

ax[0].axvspan(xlim2[0], nus150[0], color = 'k', alpha = greyscale)
ax[0].axvspan(nus220[-1], xlim2[-1], color = 'k', alpha = greyscale)
ax[2].axvspan(xlim2[0], nus150[0], color = 'k', alpha = greyscale)
ax[2].axvspan(nus220[-1], xlim2[-1], color = 'k', alpha = greyscale)

ax[0].set_xlim(xlim)
ax[0].set_ylim(ylim)
ax[0].text(xlim[0]+(xlim[1]-xlim[0])*0.1, ylim[-1]*0.8, '150 GHz \n band', fontsize = 10)
ax[0].text(xlim[0]+(xlim[1]-xlim[0])*0.6, ylim[-1]*0.8, '220 GHz \n band', fontsize = 10)
ax[2].set_xlim(xlim2)
ax[2].set_ylim(ylim2)
ax[2].text(xlim2[0]+(xlim2[1]-xlim2[0])*0.1, ylim2[-1]*0.8, '150 GHz \n band', fontsize = 10)
ax[2].text(xlim2[0]+(xlim2[1]-xlim2[0])*0.6, ylim2[-1]*0.8, '220 GHz \n band', fontsize = 10)

# Displaying maps    
plt.axes(ax[1])
auxmapG = np.sqrt(maps_ud[2, 0, :, 1] ** 2 + maps_ud[2, 0, :, 2] ** 2)
auxmapG[~cov_ud[2]] = hp.UNSEEN
hp.gnomview(auxmapG,
            reso = 15, hold = True, notext = True, 
            title = ' ',
            min = 0,
            cbar = True,
            unit = r'$\mu$K',
            rot = centers[2])
hp.projscatter(hp.pix2ang(nside_new, pixs_ud[2]),marker = '*',color = 'r', s = 180)
dpar = 10
dmer = 20
#Watch out, the names are wrong (change it)
mer_coordsG = [centers[2][0] - dmer, centers[2][0], centers[2][0] + dmer]
long_coordsG = [centers[2][1] - 2*dpar, centers[2][1] - dpar, 
                centers[2][1], centers[2][1] + dpar, centers[2][1] + 2 * dpar]
#paralels
for ilong in long_coordsG:
    plt.text(np.deg2rad(mer_coordsG[0] - 12), 1.1*np.deg2rad(ilong), 
             r'{}$\degree$'.format(ilong))
#meridians
for imer in mer_coordsG:
    if imer < 0:
        jmer = imer + 360
        ip, dp = divmod(jmer/15,1)
    else:
        ip, dp = divmod(imer/15,1)
    if imer == 0:
        plt.text(-np.deg2rad(imer + 3), np.deg2rad(long_coordsG[-1] + 6), 
             r'{}$\degree$'.format(int(ip) ))
    else:
        plt.text(-np.deg2rad(imer + 3), np.deg2rad(long_coordsG[-1] + 6), 
             r'{}$\degree$'.format(imer))
             #r'{}h{}m'.format(int(ip), int(round(dp*60))))
hp.projtext(mer_coordsG[1] + 2, long_coordsG[0] - 6, '$l$',  color = 'k', lonlat=True)
hp.projtext(mer_coordsG[2] + 12.5, long_coordsG[2] - 1, '$b$', rotation = 90, color = 'k', lonlat=True)

plt.axes(ax[3])
auxmapQ = np.sqrt(maps_ud[0, 0, :, 1] ** 2 + maps_ud[0, 0, :, 2] ** 2)
auxmapQ[~cov_ud[0]] = hp.UNSEEN
hp.gnomview(auxmapQ,
            reso = 15, hold = True, notext = True, 
            max = 7,
            min = 0,
            title = ' ',
            cbar = True,
            unit = r'$\mu$K',
            rot = centers[0])
hp.projscatter(hp.pix2ang(nside_new,pixs_ud[0]), marker = '*', color = 'r', s = 180)
mer_coordsQ = [centers[0][0] - dmer, centers[0][0]+0, centers[0][0] + dmer]
long_coordsQ = [centers[0][1] - 2*dpar, centers[0][1] - dpar, 
                centers[0][1], centers[0][1] + dpar, centers[0][1] + 2 * dpar]
#paralels
for ilong in long_coordsQ:
    plt.text(np.deg2rad(mer_coordsQ[0]-360+31), 1.1*np.deg2rad(ilong+58), r'{:.0f}$\degree$'.format(ilong),)
#meridians
for imer in mer_coordsQ:
    ip, dp = divmod(imer/15,1)
    plt.text( - np.deg2rad(imer - 360 + 48), np.deg2rad(long_coordsQ[-1] + 58 + 7), 
         r'{:.1f}$\degree$'.format(imer))

hp.graticule(dpar = dpar, dmer = dmer, alpha = 0.6, verbose = False)
#l = ax[0].legend([(p1, p2), (e1, e2)], ['Input sky', '68% C.L.'], numpoints=1, loc = 4, fontsize = 12,
#               handler_map={tuple: HandlerTuple(ndivide=None)})
l = ax[0].legend([(t0,t1), (p1, p2), (e1, e2)], ["Dust - Synch", 'Input sky', '68% C.L.'], numpoints=1, 
                 loc = 2, fontsize = 12,
               handler_map={tuple: HandlerTuple(ndivide=None)})

plt.tight_layout()#plt.tight_layout()

if savefigs: 
    plt.savefig('Figs-FI-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixQ{}_pixG{}_Polarization_grat.svg'.format(nside_new,
                                                                FuncModel.__name__,
                                                                d150Q['nf_recon'], nside_new,
                                                           pixQ_ud, pixG_ud), 
            format = 'svg', bbox_inches='tight')
    plt.savefig('Figs-FI-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixQ{}_pixG{}_Polarization_grat.pdf'.format(nside_new,
                                                                                                             FuncModel.__name__,
                                                                d150Q['nf_recon'], nside_new,
                                                           pixQ_ud, pixG_ud), 
            format = 'pdf', bbox_inches='tight')
    plt.savefig('Figs-FI-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixQ{}_pixG{}_Polarization_grat'.format(nside_new,
                                                                            FuncModel.__name__,
                                                                           d150Q['nf_recon'], 
                                                                           nside_new,pixQ_ud, pixG_ud),
           bbox_inches='tight')
else:
    plt.show()